### Import related API

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers, backend
from tensorflow.keras.layers import Dense, LSTM, Dropout, TimeDistributed, LeakyReLU, GRU, BatchNormalization
from keras.layers.core import RepeatVector
from keras.callbacks import EarlyStopping

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from itertools import product

%matplotlib inline

### Control the upper limit of GPU memory

In [3]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.3)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(sess)

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))### Read the data

In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

In [5]:
def read(path):
    return pd.read_csv(path)

In [6]:
def buildTrain(train, pastWeek=4, futureWeek=1, defaultWeek=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureWeek-pastWeek):
        X = np.array(train.iloc[i:i+defaultWeek])
        X = np.append(X,train["CCSP"].iloc[i+defaultWeek:i+pastWeek])
        X_train.append(X.reshape(X.size))
        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek]["CCSP"]))
    return np.array(X_train), np.array(Y_train)

In [43]:
def get_data():
    
    ## Read weekly copper price data
    path = "WeeklyFinalData.csv"
    data = read(path)
    
    date = data["Date"]
    data.drop("Date", axis=1, inplace=True)
    
    ## Add time lag (pastWeek=4, futureWeek=1)
    x_data, y_data = buildTrain(data)
    
    ## Data split
    x_train = x_data[0:int(x_data.shape[0]*0.8)]
    x_test = x_data[int(x_data.shape[0]*0.8):]
    
    y_train = y_data[0:int(y_data.shape[0]*0.8)]
    y_test = y_data[int(y_data.shape[0]*0.8):]
    
    ## Normalize
    x_train_scaled = sc.fit_transform(x_train)
    x_test_scaled = sc.transform(x_test)
    
    y_train_scaled = sc.fit_transform(y_train)
    y_test_scaled = sc.transform(y_test)
    
    ## Other information
    nb_output = 1
    input_shape = x_train_scaled.shape[1]
    
    return (nb_output, input_shape, x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled)

In [44]:
nb_output, input_shape, x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = get_data()

### Setting the format of the learning graph

In [45]:
def show_raw_visualization(data):

    fig, axes = plt.subplots(
        nrows=int(round(data.shape[1]/2,0)), ncols=2, figsize=(15, 5), dpi=80, facecolor="w", edgecolor="k"
    )

    for i in range(data.shape[1]):
        t_data = data.iloc[:,i]
        ax = t_data.plot(
            ax=axes[i % 2],
            color="black",
            # title="Lag:{0}, {1} curve".format(lag_time+1, data.columns[i])
        )
    
#     fig.set_size_inches(10,15)
        
    plt.tight_layout()

## Building Model

### 2-layer Neural Network

In [46]:
def buildTwoLayerNN(training_data_shape, setting):

    keras.backend.clear_session()
    regressor = Sequential()
    L2 = tf.keras.regularizers.L2(setting[4])
    regressor.add(Dense(units=setting[1], activation =setting[0], input_dim=training_data_shape, kernel_initializer = setting[2], kernel_regularizer= L2))
    regressor.add(Dense(units=1)) 

    # adam = optimizers.Adam(lr=0.0001,beta_1=0.9,beta_2=0.999, decay=1e-6)

    regressor.compile(optimizer=setting[3], loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError()])
    # regressor.summary()

    return regressor

### Performance evaluation

In [47]:
def accuracy_eva(raw_data, predict_data):
    
    total_times = raw_data.shape[0]
    accuracy_indicator = []
  ##Test
#     print(type(raw_data))
#     print(type(predict_data))

    for threshold in range(1000,3001,1000):
        correct_times = 0
        
        for i in range(raw_data.shape[0]): 

            if tf.abs(raw_data[i]-predict_data[i]) <= threshold:
                correct_times +=1

        accuracy_indicator.append(correct_times/total_times)
    
    return (accuracy_indicator)

### Training the model

In [48]:
def training_model(setting, setting_name):

    ## data
    nb_output, input_shape, x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = get_data()

    ## Training model

    #training_data_shape, activation_function, hidden_node, initializer, optimizer
    regressor = buildTwoLayerNN(input_shape, setting[1:])
    history = regressor.fit(x=x_train_scaled, y=y_train_scaled, epochs= 16, verbose=0)

    ## Draw the learning graph
    RMSE = [i for i in history.history["root_mean_squared_error"]]
    score_data = pd.DataFrame({"Loss":history.history["loss"], "RMSE":RMSE})

    ## Evaluate the model using testing data
    # scores = regressor.evaluate(data[2], data[3])  

    ##Test
    predict = regressor.predict(x_test_scaled)
    ##Test
    
#     if (np.isnan(predict).sum()>0):
#         print(setting_name)
#         score = float("inf")
#         accuracy = 0
    
#     else:

    Y_testing_data =  y_test_scaled
    

    accuracy = accuracy_eva(Y_testing_data, predict)
    
    performance = {
        "Epochs":setting_name[0],
        "Activation function":setting_name[1],
        "Hidden nodes":setting_name[2],
        "Initializer":setting_name[3],
        "Optimizer":setting_name[4],
        "Regularizer":setting_name[5],
        "In-sample RMSE": score_data["RMSE"],
        "Out-of-sample Accuracy(1000)": accuracy[0],
        "Out-of-sample Accuracy(2000)": accuracy[1],
        "Out-of-sample Accuracy(3000)": accuracy[2]
    }
    

    ##Test
    predicts = pd.DataFrame(predict)
    
    return performance, predicts.stack().reset_index(drop=True)

### Control model Hyperparameter
such as:
* input factor dimension
* epoch
* activation_function
* hidden_node
* initializer
* optimizer

In [49]:
def main():

    ## data, epoch, activation_function, hidden_node, initializer, optimizer
    ## epoches, activation_functions, hidden_nodes選項
    
    ### 0, 1, 2
    epoches = range(10, 301)
    activation_functions = ["sigmoid", "tanh", "relu"]
    hidden_nodes = range(3, 50)

    ## initializer 選項
    small_random = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
    Xavier = tf.keras.initializers.GlorotNormal()
    
    ### 3
    initializers = [small_random, Xavier]
    initializers_name = ["small_random", "Xavier"]

    # optimizer & learning rate decay
    learning_rate_fn = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate = 0.01, 
        decay_steps = 10,
        alpha = 0.01
    )

    SGD_cosine = tf.keras.optimizers.SGD(
      learning_rate=learning_rate_fn,
      name = "SGD_with_cosine"
    )

    Adam_cosine = tf.keras.optimizers.Adam(
      learning_rate=learning_rate_fn,
      name = "Adam_with_cosine"
    )

    Momentum = tf.keras.optimizers.SGD(
      learning_rate=0.01,
      momentum=0.9,
      name = "Momentum"
    )

    Mom_cosine = tf.keras.optimizers.SGD(
      learning_rate=learning_rate_fn,
      momentum=0.9,
      name = "Momentum_with_cosine"
    )
    
    ## optimizers 選項
    ### 4
    optimizers = ["SGD", SGD_cosine, "Adam", Adam_cosine, Momentum, Mom_cosine]
    optimizers_name = ["SGD", "SGD_with_cosine", "Adam", "Adam_with_cosine", "Momentum", "Momentum_with_cosine"]

    ## regularizers 選項
    ### 5
    regularizer= np.arange(0,0.0011, 0.0001)
    
    ##Test
#     optimizers = ["SGD","Adam"]
#     optimizers_name = ["SGD","Adam"]

    ##所有hyperparameter的組合
    sets = list(product(epoches, activation_functions, hidden_nodes, initializers, optimizers, regularizer))
    sets_name = list(product(epoches, activation_functions, hidden_nodes, initializers_name, optimizers_name, regularizer))

    ### Performance indicator
    RMSE_scores = pd.DataFrame(columns = ["Epochs", "Activation function", "Hidden nodes", "Initializer", "Optimizer", "Regularizer","In-sample RMSE", "Out-of-sample Accuracy(1000)", "Out-of-sample Accuracy(2000)", "Out-of-sample Accuracy(3000)"])
    predicts = pd.DataFrame()
    
    for i in range(len(sets)):
        
        performance, predict = training_model(sets[i], sets_name[i])
        
        RMSE_scores = RMSE_scores.append(performance, ignore_index=True)
        predicts = predicts.append(predict, ignore_index=True)

    return RMSE_scores, predicts

In [ ]:
if __name__ == "__main__":

    RMSE_scores, predicts = main()

### The best top five models based on in-sample data

In [ ]:
RMSE_scores.sort_values("In-sample RMSE", inplace=True)
RMSE_scores.head(5)

### Ensemble the top 5 models

In [ ]:
top_five_model = predicts.iloc[list(RMSE_scores.head(5).index),:]
ensemble_predict = top_five_model.apply(lambda x: x.mean())
ensemble_RMSE = np.sqrt(mean_squared_error(ensemble_predict, data[3]))
emsemble_accuracy = accuracy_eva(data[3], ensemble_predict, 3000)
print("RMSE of ensembling the top 5 models: %.5f" %ensemble_RMSE)
print("Accuracy of ensembling the top 5 models: %.5f" %emsemble_accuracy)

plt.figure(figsize=(10,4))
plt.scatter(range(data[3].shape[0]), data[3], color='black', label = 'Raw data')
plt.plot(ensemble_predict, label = 'Predicted value')
plt.legend()
plt.xlabel("Sample index (weekly)")
plt.ylabel("Cooper price value")
plt.savefig('Result_ensemble_model.png')

### Save all the results of model configuration in in-sample data as Settings_RMSE_scores.csv

In [ ]:
RMSE_scores.reset_index(inplace=True, drop=True)
RMSE_scores.to_csv("Settings_score_DINKLE.csv", index=False)
RMSE_scores

### The best configuration in out-of-sample data

In [ ]:
RMSE_scores.sort_values("Out-of-sample RMSE", inplace=True)
RMSE_scores.reset_index(inplace=True, drop=True)
RMSE_scores[RMSE_scores["Out-of-sample RMSE"]==min(RMSE_scores["Out-of-sample RMSE"])]